In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import cPickle as pickle

In [2]:
#import rdkit
import rdkit.Chem.AllChem
#import rdkit.Chem.Draw

In [3]:
from collections import Counter

In [4]:
%%time
df_train = pd.read_csv("train.csv")

Wall time: 1min 22s


In [5]:
%%time
temp = df_train.sample(n=100000) # df_train.sample(frac=0.1)
del df_train
df_train = temp
del temp

Wall time: 1.84 s


In [6]:
orig_feature_names = list(df_train.columns[1:(len(df_train.columns)-1)])
feature_names = list(df_train.columns[1:(len(df_train.columns)-1)])

In [7]:
%%time
df_train['molecule'] = map(rdkit.Chem.MolFromSmiles, df_train['smiles'].values)

Wall time: 1min 41s


In [8]:
%%time
df_train['atom_count'] = map(lambda m : len(m.GetAtoms()), df_train['molecule'].values)
feature_names.append('atom_count')

Wall time: 562 ms


In [9]:
%%time
df_train['len'] = map(len, df_train['smiles'].values)
feature_names.append('len')

Wall time: 95 ms


In [10]:
%%time
df_train['equals'] = map(lambda s : s.count('='), df_train['smiles'].values)
feature_names.append('equals')

Wall time: 149 ms


In [11]:
%%time
df_train['double'] = map(lambda m : sum(np.array([b.GetBondType() for b in m.GetBonds()])==2), df_train['molecule'])
feature_names.append('double')

Wall time: 32.5 s


In [12]:
%%time
df_train['triple'] = map(lambda m : sum(np.array([b.GetBondType() for b in m.GetBonds()])==3), df_train['molecule'])
feature_names.append('triple')

Wall time: 32.8 s


In [13]:
%%time
max_atoms = max(df_train['atom_count'].values)
for r in range(max_atoms):
    df_train['ring '+str(r)] = map(lambda m : m.GetRingInfo().NumAtomRings(r), df_train['molecule'])
    feature_names.append('ring '+str(r))

Wall time: 13.5 s


In [14]:
%%time
element_count = {element : [] for element in range(1,151)}
for m in df_train['molecule']:
    counts = Counter([atom.GetAtomicNum() for atom in m.GetAtoms()])
    for element in range(1,151):
        element_count[element].append(counts[element])

Wall time: 29.1 s


In [15]:
%%time
elements = [k for k in element_count if sum(element_count[k]) > 0]
print elements
for element in elements:
    df_train['count '+str(element)] = element_count[element]
    feature_names.append('count '+str(element))

[6, 7, 8, 14, 16, 34]
Wall time: 309 ms


In [ ]:
# commented out because this is just the sum of the 50 columns above, so it creates linear dependence and doesn't add information
#%%time
#max_atoms = max(df_train['atom_count'].values)
#df_train['ring total'] = map(lambda m : sum([m.GetRingInfo().NumAtomRings(r) for r in range(max_atoms)]), df_train['molecule'])
#feature_names.append('ring total')

In [29]:
#df_train['bond_count'] = map(lambda s : len(rdkit.Chem.MolFromSmiles(s).GetBonds()), df_train['smiles'].values)
#feature_names.append('bond_count')

In [18]:
df_train = df_train.reset_index(drop=True)

In [19]:
%%time
validation_indices = np.random.choice(range(len(df_train)), size=int(0.3*len(df_train)), replace=False)
mask = np.array([False for n in range(len(df_train))])
mask[validation_indices] = True
df_valid = df_train[mask]
df_train = df_train[~mask]
print df_valid.shape
print df_train.shape

(30000, 305)
(70000, 305)
Wall time: 353 ms


In [22]:
%%time
X_train_old = df_train[orig_feature_names].values
X_train = df_train[feature_names].values
Y_train = df_train['gap'].values

Wall time: 949 ms


In [24]:
%%time
X_valid_old = df_valid[orig_feature_names].values
X_valid = df_valid[feature_names].values
Y_valid = df_valid['gap'].values

Wall time: 386 ms


In [25]:
%%time
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred_valid = LR.predict(X_valid)
#rmse = reduce(lambda s, (a,b) : s + (a-b)**2, zip(Y_train, LR_pred_train), 0)
rmse = mean_squared_error(Y_valid, LR_pred_valid)
print rmse

0.0463584922673
Wall time: 56.5 s


In [26]:
%%time
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred_valid = RF.predict(X_valid)
#rmse = reduce(lambda s, (a,b) : s + (a-b)**2, zip(Y_train, RF_pred_train), 0)
rmse = mean_squared_error(Y_valid, RF_pred_valid)
print rmse

0.0272790916525
Wall time: 17.2 s


In [27]:
%%time
RR = RidgeCV(alphas = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000])
#RR.fit(np.vstack((X_train,X_valid)), np.hstack((Y_train,Y_valid)))
RR.fit(X_train, Y_train)
print 'Optimal alpha:', RR.alpha_
RR_pred_valid = RR.predict(X_valid)
#rmse = reduce(lambda s, (a,b) : s + (a-b)**2, zip(Y_train, RR_pred_train), 0)
rmse = mean_squared_error(Y_valid, RR_pred_valid)
print rmse
print RR.coef_

Optimal alpha: 1e-06
0.0463652622934
[ 0.01658295  0.          0.          0.          0.         -0.02793089
  0.37444477  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.01996552  0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.06793164  0.          0.          0.          0.
  0.          0.          0.2306568   0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.03120602
 -0.06337797  0.          0.          0.00759286  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.       

In [38]:
%%time
print {alpha : mean_squared_error(Y_valid, Ridge(alpha = alpha).fit(X_train, Y_train).predict(X_valid))
       for alpha in [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

{0: 0.046396924141956973, 1: 0.046354876619818242, 1e-05: 0.046366289171508153, 100: 0.046394693927244969, 0.1: 0.046362775957202326, 10: 0.04635260072841204, 0.001: 0.046366244980778598, 0.0001: 0.046366285144420136, 0.01: 0.046365853804375703}
Wall time: 10.2 s


In [37]:
%%time
print {alpha : mean_squared_error(Y_valid, Lasso(alpha = alpha).fit(X_train, Y_train).predict(X_valid))
       for alpha in [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

{0: 0.04635848670080471, 1: 0.12894090170895284, 1e-05: 0.046354366090614274, 100: 0.16575261505023134, 0.1: 0.10113925992349254, 10: 0.16575261505023134, 0.001: 0.047382547990575528, 0.0001: 0.046389688170792566, 0.01: 0.058688458380508263}
Wall time: 51.8 s


C:\Users\Madhu\Anaconda\envs\my-rdkit-env\lib\site-packages\ipykernel\__main__.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  from ipykernel import kernelapp as app


In [29]:
%%time
rmsedict = dict()
for alpha in [0, 0.1, 1, 10]:
    L = Lasso(alpha=alpha)
    L.fit(X_train, Y_train)
    rmsedict[alpha] = mean_squared_error(Y_valid, L.predict(X_valid))
print rmsedict

{0: 0.04635848670080471, 1: 0.12894090170895284, 10: 0.16575261505023134, 0.1: 0.10113925992349254}
Wall time: 10.6 s


C:\Users\Madhu\Anaconda\envs\my-rdkit-env\lib\site-packages\ipykernel\__main__.py:4: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator


In [ ]:
%%time
LassoReg = LassoCV(alphas = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000])
RR.fit(np.vstack((X_train,X_valid)), np.hstack((Y_train,Y_valid)))
#RR.fit(X_train, Y_train)
print LassoReg.alpha_
Lasso_pred_valid = LassoReg.predict(X_valid)
#rmse = reduce(lambda s, (a,b) : s + (a-b)**2, zip(Y_train, RR_pred_train), 0)
rmse = mean_squared_error(Y_valid, Lasso_pred_valid)
print rmse
print LassoReg.coef_

In [53]:
%%time
EN = ElasticNetCV(l1_ratio = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
                  alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], cv = 5)
EN.fit(X_train, Y_train)
print EN.alpha_
print EN.l1_ratio_
print mean_squared_error(Y_valid, EN.predict(X_valid))

0.0001
0.0
0.0463524896982
Wall time: 1min 3s


In [26]:
mol = rdkit.Chem.MolFromSmiles(df_train['smiles'][df_train.index[0]])

In [36]:
rdkit.Chem.Draw.ShowMol(mol)

ImportError: No module named PIL

In [81]:
print [atom.GetAtomicNum() for atom in mol.GetAtoms()]
sum([bond.GetBondType() == rdkit.Chem.rdchem.BondType.SINGLE for bond in mol.GetBonds()])

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 16, 7, 6, 6, 6, 6, 6, 6, 6, 8, 6, 6]


1

In [3]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [7]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [20]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [6]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [50]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [45]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


In [51]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (1000000, 256)
Train gap: (1000000,)
Test features: (824230, 256)


In [38]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [47]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [40]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [48]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)